# Binary segmentation with the Lovász Hinge
## Tensorflow version, see PyTorch version for more details & use in training

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import division, print_function

In [ ]:
import sys

In [ ]:
import tensorflow as tf

In [ ]:
%run ../src/lovasz.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
"""
Some utils used in tf demos
Maxim Berman 2018 ESAT-PSI KU Leuven (MIT License)
"""

from __future__ import print_function, division
import tensorflow as tf

# https://danijar.com/structuring-your-tensorflow-models/

import functools

def doublewrap(function):
    """
    A decorator decorator, allowing to use the decorator to be used without
    parentheses if not arguments are provided. All arguments must be optional.
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator


@doublewrap
def define_scope(function, scope=None, *args, **kwargs):
    """
    A decorator for functions that define TensorFlow operations. The wrapped
    function will only be executed once. Subsequent calls to it will directly
    return the result so that operations are added to the graph only once.
    The operations added by the function live within a tf.variable_scope(). If
    this decorator is used with arguments, they will be forwarded to the
    variable scope. The scope name defaults to the name of the wrapped
    function.
    """
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
        return getattr(self, attribute)
    return decorator

"""
Some utils used in all demos
Maxim Berman 2018 ESAT-PSI KU Leuven (MIT License)
"""

import numpy as np
from PIL import Image, ImageDraw
import contextlib


def paletteVOC(N=256, normalized=False, PIL=False):
    """
    Pascal VOC color map
    """
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)

    dtype = 'float32' if normalized else 'uint8'
    cmap = np.zeros((N, 3), dtype=dtype)
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7-j)
            g = g | (bitget(c, 1) << 7-j)
            b = b | (bitget(c, 2) << 7-j)
            c = c >> 3

        cmap[i] = np.array([r, g, b])

    cmap = cmap/255 if normalized else cmap
    if PIL:
        cmap = [k for l in cmap for k in l]
    return cmap


def pil(array):
    im = Image.fromarray(array)
    im.putpalette(paletteVOC(PIL=True))
    return im


def pil_grid(images, max_horiz=np.iinfo(int).max, margin=0, background='white'):
    """
    Grid of images in PIL
    """
    n_images = len(images)
    n_horiz = min(n_images, max_horiz)
    h_sizes, v_sizes = [0] * n_horiz, [0] * (n_images // n_horiz)
    for i, im in enumerate(images):
        h, v = i % n_horiz, i // n_horiz
        h_sizes[h] = max(h_sizes[h], im.size[0]) + margin
        v_sizes[v] = max(v_sizes[v], im.size[1]) + margin
    h_sizes, v_sizes = np.cumsum([0] + h_sizes), np.cumsum([0] + v_sizes)
    im_grid = Image.new('RGB', (h_sizes[-1], v_sizes[-1]), color=background)
    for i, im in enumerate(images):
        im_grid.paste(im, (h_sizes[i % n_horiz], v_sizes[i // n_horiz]))
    return im_grid


def dummy_triangles(w, categories=[0, 255, 1]):
    """
    Generate random images with desired categories and random triangles
    """
    im = Image.new('P', (w, w), color=categories[0])
    im.putpalette(paletteVOC(PIL=True))
    draw = ImageDraw.Draw(im)
    for c in categories[1:]:
        draw.polygon([tuple(p) for p in np.random.randint(w, size=(3, 2))], fill=c, outline=None)
    return im


# https://stackoverflow.com/questions/2891790/how-to-pretty-printing-a-numpy-array-without-scientific-notation-and-with-given
@contextlib.contextmanager
def printoptions(*args, **kwargs):
    original = np.get_printoptions()
    np.set_printoptions(*args, **kwargs)
    try:
        yield
    finally:
        np.set_printoptions(**original)

### Ground truth

In [ ]:
H = W = 200
IGNORE = 255
classes = [0, 255, 1]
C = len([c for c in classes if c != IGNORE])
B = 2

In [ ]:
np.random.seed(18)
labels_ = [dummy_triangles(H, classes) for b in range(B)]
labels = np.stack(map(np.array, labels_)).astype(np.uint8)
pil_grid(labels_, 5, margin=1)

### Features

In [ ]:
np.random.seed(57)
feats = np.zeros(labels.shape)
feats[labels == 0] = np.random.normal(-1, 2.5, size=feats.shape)[labels == 0]
feats[labels == 1] = np.random.normal(1, 2.5, size=feats.shape)[labels == 1]
feats[labels == 255] = np.random.normal(0, 5, size=feats.shape)[labels == 255]

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
labels_tf = tf.placeholder(np.int64, shape=(None, H, W), name='labels')
feats_tf = tf.placeholder(np.float32, shape=(None, H, W), name='features')

In [ ]:
feed_dict = {labels_tf: labels, 
             feats_tf: feats}

### Model

In [ ]:
class Model:
    '''
    Simple linear model
    '''
    
    def __init__(self, feats):
        self.feats = feats
        self.scores
        self.predict

    @define_scope
    def scores(self):
        x = self.feats
        bias = tf.Variable(0., x.dtype, name="bias")
        return tf.add(x, bias, name="scores")
    
    @define_scope
    def probas(self):
        return tf.nn.sigmoid(self.scores)
    
    @define_scope
    def predict(self):
        return tf.greater_equal(self.scores, 0, name="predict")

In [ ]:
m = Model(feats_tf)

In [ ]:
sess.run(tf.global_variables_initializer())

## Lovász-Hinge value

In [ ]:
loss = lovasz_hinge(m.scores, labels_tf, ignore=IGNORE, per_image=True)

In [ ]:
loss.eval(feed_dict)

#### Gradient w.r.t. bias

In [ ]:
tf.gradients(loss, tf.trainable_variables()[0])[0].eval(feed_dict)

### PyTorch reference

In [ ]:
list(labels[1])

should give the same results...

In [ ]:
sess.run(m.probas, feed_dict = feed_dict)

In [ ]:
#[-1, 200, 200]
#[-1, 200,2 00]

## Lovász-Sigmoid value

In [ ]:
loss = lovasz_softmax(m.probas, labels_tf, classes=[1], ignore=IGNORE, per_image=True)

In [ ]:
loss.eval(feed_dict)

In [ ]:
tf.gradients(loss, tf.trainable_variables()[0])[0].eval(feed_dict)

### PyTorch reference

In [ ]:
sigmoid_out = torch.sigmoid(out)

In [ ]:
loss_torch = L_pytorch.lovasz_softmax(sigmoid_out, torch.from_numpy(labels), classes=[1], ignore=IGNORE, per_image=True)
loss_torch

In [ ]:
m_torch.bias.grad.data.fill_(0)
loss_torch.backward()

In [ ]:
m_torch.bias.grad.data.numpy().item()